# Notes

Reason for using Selenium

The job search results page keeps adding more results at the bottom, but the URL stays the same. Therefore, it is not possible to loop through the pages. It is necessary to click on the next button until all the results are loaded and then scrape.

The url for a job search results page (for example for technology), when used later, redirects to the job search home page
without the search results. Therefore, it is necessary to type in the search term do a fresh search before scraping.

To do for this notebook

- see if just the qualifications part can be scraped.

Running the script 
    
- It takes about 10-15 minutes to run the part that scrapes the job titles and job description page links.
- It may take 3-4 hours to run the part that scrapes the job description text. It may take longer 
  if the script gets interruped for some reason. It is necessary for a person to check if the script run successfully 
  and rerun as needed.
- It looks like the script tends to get interrupted if the computer goes to sleep mode. 
  So it helps to keep the computer active for the script to complete running without interruption.

In [25]:
from selenium import webdriver
import pandas as pd
import numpy as np

# Get job titles and job description page links

In [3]:
# job_title = []
# job_link = []

#this url redirects to the page 'https://sjobs.brassring.com/TGnewUI/Search/Home/Home?partnerid=25633&siteid=5439&Codes=BeMore#home'
#url = 'https://sjobs.brassring.com/TGnewUI/Search/Home/Home?partnerid=25633&siteid=5439&Codes=BeMore#keyWordSearch=technology%20or%20software%20engineering%20or%20developer%20or%20azure%20or%20aws&locationSearch='

url = 'https://sjobs.brassring.com/TGnewUI/Search/Home/Home?partnerid=25633&siteid=5439&Codes=BeMore#home'

driver = webdriver.Chrome()

driver.implicitly_wait(20)

driver.get(url)
    
location = driver.find_element_by_xpath('//*[@id="initialSearchBox__26"]')
location.send_keys('usa')

search_button = driver.find_element_by_xpath('//*[@id="searchControls_BUTTON_2"]')
search_button.click()



In [4]:
x = 1

while x == 1:
    try:
    
        next_button = driver.find_element_by_xpath('//*[@id="showMoreJobs"]')
        next_button.click()

    except:
        x = 0    

In [5]:
job_title = []
job_link = []

In [6]:

y = 2
i = 1

while y != 0:  
    
    y = 2
    
    try:  
        job = driver.find_element_by_xpath('//*[@id="mainJobListContainer"]/div/div/ul/li['+str(i)+']/div[2]/div[1]')        
        job_title.append(job.text)
        
    except:
        job_title.append('')
        y -= 1

        
    try:
        link = driver.find_element_by_xpath('//*[@id="Job_'+str(i)+'"]')        
        job_link.append(link.get_attribute('href'))
        
    except:
        job_link.append('')
        y -= 1        

    i = i+1
    
    
print(len(job_title), len(job_link)) 
    


597 597


In [7]:
# convert to dataframe

df = pd.DataFrame(zip(job_title, job_link))
df.columns = ['TITLE', 'LINK']
df.head()

,TITLE,LINK
0,Senior Consultant - Digital Product Management...,https://sjobs.brassring.com/TGnewUI/Search/hom...
1,Lead - JAVA/Selenium/BDD/API/Mobility - Charlo...,https://sjobs.brassring.com/TGnewUI/Search/hom...
2,"Oracle Functional Consultant - ORMB - Chicago,...",https://sjobs.brassring.com/TGnewUI/Search/hom...
3,D365 F&O SCM Consultant,https://sjobs.brassring.com/TGnewUI/Search/hom...
4,Medicaid Business Analyst,https://sjobs.brassring.com/TGnewUI/Search/hom...


In [8]:
df['QUALIFICATIONS'] = np.nan
df.head()

,TITLE,LINK,QUALIFICATIONS
0,Senior Consultant - Digital Product Management...,https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN
1,Lead - JAVA/Selenium/BDD/API/Mobility - Charlo...,https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN
2,"Oracle Functional Consultant - ORMB - Chicago,...",https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN
3,D365 F&O SCM Consultant,https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN
4,Medicaid Business Analyst,https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN


In [9]:
df.to_csv('infosys_jobs_usa_title_link.csv')

In [3]:
df = pd.read_csv('infosys_jobs_usa_title_link.csv')
df = df.drop(columns=['Unnamed: 0'], axis=1)
df.head()

,TITLE,LINK,QUALIFICATIONS
0,Senior Consultant - Digital Product Management...,https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN
1,Lead - JAVA/Selenium/BDD/API/Mobility - Charlo...,https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN
2,"Oracle Functional Consultant - ORMB - Chicago,...",https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN
3,D365 F&O SCM Consultant,https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN
4,Medicaid Business Analyst,https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN


# Get job description data

In [ ]:
# job description text - wording and how the text and the headings are organized differs among job postings
# the most commonality I could find is most of the job descriptions and qualifications are under li tags, 
# but not always. Therefore, I am grabbing all li tags from the job description pages.

In [ ]:
# get job description data in batches. It takes too long to get all at once.

In [4]:
driver = webdriver.Chrome()   
    
for i in range(len(df['LINK'])):
    
    try:
        job_text = ''
        url = (df['LINK'][i])

        driver.get(url)

        desc = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[7]/div[4]/div[2]/div/div[3]/div[4]/p[2]')
        texts = desc.find_elements_by_tag_name('li')

        for Text in texts: 
            job_text = job_text+Text.text+' '

        df['QUALIFICATIONS'][i] = job_text

    except:
        df['QUALIFICATIONS'][i] = ''

driver.close()

C:\Users\meske\AppData\Local\Temp/ipykernel_31628/2589495480.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['QUALIFICATIONS'][i] = job_text


In [8]:
df['QUALIFICATIONS'][500]

'Transformation Programs Global, Template, Roll-out Programs SAP S/4 HANA Transformation or Migration Programs Support Infosys Consulting leadership team members in driving proposal development and leading proposal discussions Develop client relationships to identify and qualify sales opportunities with the assistance of senior leadership Lead solution definition for client specific needs during opportunity to closure cycle Articulate our unique point of view in various thought leadership media including papers, webinars and seminars Collaborate with clients to define long-term vision, goals and strategies Contribute to the design and development of client future processes and organizations with corresponding transformation roadmaps Conduct requirements workshops with a team of global IT professional & subject matter experts, translating those requirements into actionable project initiatives w/associated metrics Solution targeted processes, leveraging best practices Advise, QA and prov

In [9]:
len(df[df['QUALIFICATIONS'] == ''])


198

# Basic Cleaning

In [14]:
to_drop = df[df['QUALIFICATIONS'] == ''].index
df = df.drop(to_drop)
df = df.dropna()
len(df)


399

In [16]:
df['QUALIFICATIONS'] = df['QUALIFICATIONS'].str.lower()

In [17]:
df = df.drop_duplicates(subset=['TITLE', 'QUALIFICATIONS'])
len(df)

389

In [18]:
df = df.reset_index(drop=True)

In [19]:
df.head()

,TITLE,LINK,QUALIFICATIONS
0,"Oracle Functional Consultant - ORMB - Chicago,...",https://sjobs.brassring.com/TGnewUI/Search/hom...,bachelor's degree or foreign equivalent requir...
1,D365 F&O SCM Consultant,https://sjobs.brassring.com/TGnewUI/Search/hom...,analyze federal and state regulations for medi...
2,Medicaid Business Analyst,https://sjobs.brassring.com/TGnewUI/Search/hom...,bachelor’s degree or foreign equivalent requir...
3,Senior AML Consultant,https://sjobs.brassring.com/TGnewUI/Search/hom...,bachelor’s degree or foreign equivalent requir...
4,Senior RPA Developer,https://sjobs.brassring.com/TGnewUI/Search/hom...,candidate can work from any of the infosys off...


In [21]:
df['COMPANY'] = 'Infosys'
df['DESCRIPTION'] = df['QUALIFICATIONS']
df.head()

,TITLE,LINK,QUALIFICATIONS,COMPANY,DESCRIPTION
0,"Oracle Functional Consultant - ORMB - Chicago,...",https://sjobs.brassring.com/TGnewUI/Search/hom...,bachelor's degree or foreign equivalent requir...,Infosys,bachelor's degree or foreign equivalent requir...
1,D365 F&O SCM Consultant,https://sjobs.brassring.com/TGnewUI/Search/hom...,analyze federal and state regulations for medi...,Infosys,analyze federal and state regulations for medi...
2,Medicaid Business Analyst,https://sjobs.brassring.com/TGnewUI/Search/hom...,bachelor’s degree or foreign equivalent requir...,Infosys,bachelor’s degree or foreign equivalent requir...
3,Senior AML Consultant,https://sjobs.brassring.com/TGnewUI/Search/hom...,bachelor’s degree or foreign equivalent requir...,Infosys,bachelor’s degree or foreign equivalent requir...
4,Senior RPA Developer,https://sjobs.brassring.com/TGnewUI/Search/hom...,candidate can work from any of the infosys off...,Infosys,candidate can work from any of the infosys off...


In [22]:
df.to_csv('infosys_usa_ jobs.csv')

In [28]:
df = pd.read_csv('infosys_usa_ jobs.csv')
df = df.reindex(columns=['COMPANY', 'TITLE', 'QUALIFICATIONS', 'LINK', 'DESCRIPTION'])
df.head()

,COMPANY,TITLE,QUALIFICATIONS,LINK,DESCRIPTION
0,Infosys,"Oracle Functional Consultant - ORMB - Chicago,...",bachelor's degree or foreign equivalent requir...,https://sjobs.brassring.com/TGnewUI/Search/hom...,bachelor's degree or foreign equivalent requir...
1,Infosys,D365 F&O SCM Consultant,analyze federal and state regulations for medi...,https://sjobs.brassring.com/TGnewUI/Search/hom...,analyze federal and state regulations for medi...
2,Infosys,Medicaid Business Analyst,bachelor’s degree or foreign equivalent requir...,https://sjobs.brassring.com/TGnewUI/Search/hom...,bachelor’s degree or foreign equivalent requir...
3,Infosys,Senior AML Consultant,bachelor’s degree or foreign equivalent requir...,https://sjobs.brassring.com/TGnewUI/Search/hom...,bachelor’s degree or foreign equivalent requir...
4,Infosys,Senior RPA Developer,candidate can work from any of the infosys off...,https://sjobs.brassring.com/TGnewUI/Search/hom...,candidate can work from any of the infosys off...


In [29]:
df.to_csv('infosys_usa_ jobs.csv')